# `x_model_prediction.py` 
## Generate a prediction from the x model


In [0]:
import os.path
import h5py
import pandas as pd
import numpy as np
%matplotlib inline

from tensorflow.python.keras.models import load_model

In [6]:
# TODO: add the argparse block


# Load the SavedModel
file_model = '/Users/michaeldolce/Development/files/h5-files/models/X_FHC_Model.h5'
# Locations on the WSU HPC in Abdul's directory...
# '/home/m962g264/wsu_Nova_Vertexer/output/h5_models/model_082023/X_FHC_Model.h5'
# '/home/m962g264/wsu_Nova_Vertexer/output/h5_models/model_082023/X_RHC_Model.h5'

model = load_model(file_model)

# Load the designated test file. This is file 27
# NOTE: there is only one test file for the FD validation, and it should be within the "test" directory
# ANOTHER NOTE: the test file has not been pre-processed, so it has all information within it! So can investigate deeper the interactions from each vertex.  


In [8]:
# the files in this directory are ready to be used for making predictions.
test_path = '/home/k948d562/output/trained-models/make-predictions' 
for filename in test_path:
    if args.detector and args.horn and args.flux in filename:
        print('found validation file: ', filename)
        test_path += '/' + filename
    continue


Test files validation path: 	/Users/michaeldolce/Development/files/h5-files/train-data/
Files in the test directory: 	1


In [9]:
# Checking the size of the test file(s)
      
test_files = [n for n in os.listdir(test_path) if n.endswith(".h5")] # un-used
events=0
for h5_filename in os.listdir(test_path):
    events += len((h5py.File(test_path + h5_filename, 'r')['run'][:]))
    
print('Number of validation files:', len(os.listdir(test_path)))
print('Number of validation events:', events)

Number of validation files: 1
Number of validation events: 78


In [10]:
f=h5py.File(test_path + os.listdir(test_path)[0],'r') # reading the first file in the directory
print(list(f.keys()))

['E', 'Mode', 'PDG', 'TrueRecoVtxX', 'TrueRecoVtxY', 'TrueRecoVtxZ', 'cvnmap', 'cycle', 'event', 'firstplane', 'isCC', 'lastcellx', 'lastcelly', 'run', 'slice', 'subrun']


In [11]:
# for reading all test files
idx_event = 0
test_cvnmap, test_mode, test_iscc, E, pdg, true_reco_vtx_x = [], [], [], [], [], []
# truerecovtxy, truerecovtxz -- not used here, X only.

for h5_filename in os.listdir(test_path):
    if os.path.isdir(h5_filename): #skipping directories in the files
        continue
    
    print('Processing... {} of {}'.format(idx_event, len(os.listdir(test_path))), end="\r", flush=True)
    
    with h5py.File(test_path + h5_filename, 'r') as idx_file:
        test_cvnmap.append(idx_file['cvnmap'][:])
        test_mode = np.append(test_mode, idx_file['Mode'][:], axis=0)
        test_iscc = np.append(test_iscc, idx_file['isCC'][:], axis=0)
        true_reco_vtx_x.append(idx_file['TrueRecoVtxX'][:])
        E = np.append(E, idx_file['E'][:], axis=0)
        pdg = np.append(pdg, idx_file['PDG'][:], axis=0)
        
    
    idx_event += 1

# Convert to NumPy arrays
true_reco_vtx_x = np.array(true_reco_vtx_x)


print('Test files read successfully and packed into arrays', flush=True)

Test files read successfully and packed into arrays


In [12]:
# Some simple manipulation to see what's within the files...

# print out this number of the last entries...
print_count = input( "Enter number of entries to print (from the back): ")

# print(test_mode)
# print('------')
# print(test_iscc)

# elect to print out info from these two vars...
dict_vars = {'Energy (GeV)' : E, 'PDG code' : pdg}

for dict_pair in dict_vars.items():
    print(' {} : {} '.format(dict_pair[0], dict_pair[1][-int(print_count):]))

# this is the identical as above...
# print ('last entry: ', dict_vars['Energy (GeV)'][-print_count:])



 Energy (GeV) : [3.14603209 1.40381277] 
 PDG code : [14. 14.] 


# Normalizing pixel maps via sklearn preprocessing

In [13]:
# True vertex extraction for analysis
true_vtx_x, reco_vtx_x = ([] for i in range(2))
idx_file = 0

while idx_file < len(os.listdir(test_path)):
    print('Processing...', end="\r", flush=True)
    event = 0
    
    # don't quite understand this here...
    while event < true_reco_vtx_x[idx_file].shape[0]:
        true_vtx_x = np.append(true_vtx_x, true_reco_vtx_x[idx_file][event][0])
        reco_vtx_x = np.append(reco_vtx_x, true_reco_vtx_x[idx_file][event][1])
        event += 1
    
    idx_file += 1

print('Testing preprocessing complete\n', end="\r", flush=True)

# convert to np arrays
true_vtx_x=np.array(true_vtx_x)
reco_vtx_x=np.array(reco_vtx_x)

print('True vertex x shape: {} , {}'.format(true_vtx_x.shape, true_vtx_x.ndim))
#print(true_vtx_x)


Testing preprocessing complete
True vertex x shape: (78,) , 1


In [14]:
from sklearn import preprocessing

idx_event = idx_file = 0
test_cvnmap_norm = []

print('Number of files: {}'.format(len(os.listdir(test_path))))

# Normalize the pixel map from test_cvnmap array from each file...
while idx_file < (len(os.listdir(test_path))):
    test_cvnmap_norm.append(preprocessing.normalize(test_cvnmap[idx_file], axis=1))
    idx_file+=1
# convert to np array
test_cvnmap_norm=np.array(test_cvnmap_norm)
print('Test files normalized successfully', flush=True)
# print(test_cvnmap_norm[-1:])
print(test_cvnmap_norm.shape)
print(test_cvnmap_norm.ndim)

Number of files: 1
Test files normalized successfully
(1, 78, 16000)
3


In [15]:
# split normalized cvnmap into reshaped events with multi-views
# todo: do not really understand this...
c, d, test_cvnmap_norm_resh, test_cvnmap_norm_resh_xz, test_cvnmap_norm_resh_yz = ([] for i in range(5))
idx_file, event = 0, 0

while idx_file < len(os.listdir(test_path)):
    print('Processing tests cvnmap file {} of {}\n'.format(idx_file + 1, len(os.listdir(test_path))), end="\r", flush=True)
    c = test_cvnmap_norm[idx_file]
    if idx_file == 0: 
        print(c.shape)
        print(c.shape[0])
    event = 0
    
    while event < c.shape[0]:
        d = c[event].reshape(2, 100, 80)
        test_cvnmap_norm_resh.append(d)
        test_cvnmap_norm_resh_xz.append(d[0])
        test_cvnmap_norm_resh_yz.append(d[1])
        event += 1
    
    idx_file += 1

print('ncvnmap processing complete')

test_cvnmap_norm_resh_xz=np.array(test_cvnmap_norm_resh_xz) # xz views only
test_cvnmap_norm_resh_yz=np.array(test_cvnmap_norm_resh_yz) # yz views only

print(test_cvnmap_norm_resh_xz.shape, test_cvnmap_norm_resh_xz.ndim)



Processing tests cvnmap file 1 of 1
(78, 16000)
78
ncvnmap processing complete
(78, 100, 80) 3


**np array conversion for test preparation**

In [16]:
# add one more dimension to let the CNN know we are dealing with one color dimension
test_cvnmap_norm_resh_xz=test_cvnmap_norm_resh_xz.reshape(test_cvnmap_norm_resh_xz.shape[0],100,80,1)
test_cvnmap_norm_resh_yz=test_cvnmap_norm_resh_yz.reshape(test_cvnmap_norm_resh_yz.shape[0],100,80,1)
# batch_size (you mean events?), width, height , color_channels

In [17]:
# import tensorflow as tf

# model = tf.distribute.DistributedDataset()

# todo: what is going on here? is information missing from the test files? (e.g. the Y and Z views?)
full_predictions=model.predict([test_cvnmap_norm_resh_xz, test_cvnmap_norm_resh_yz])


#Saving full Prediction CSV file
df_true_x_vtx=pd.DataFrame(true_vtx_x, columns=['True X'])
df_sub_true_x_vtx = df_true_x_vtx.iloc[:10, :10]  # Select the first 10 rows and first 10 columns
df_reco_x_vtx=pd.DataFrame(reco_vtx_x, columns=['Reco X'])
df_sub_reco_x_vtx= df_reco_x_vtx.iloc[:10, :10]
full_predictions=pd.Series(full_predictions.reshape(len(true_vtx_x), ))
df_true_x_vtx=pd.concat([df_true_x_vtx, full_predictions], axis=1)
subset_concat_df = df_true_x_vtx.iloc[:10, :10]
df_true_x_vtx=pd.concat([df_true_x_vtx, df_reco_x_vtx], axis=1)
subset_concate_df = df_true_x_vtx.iloc[:10, :10]
df_true_x_vtx.columns=['True X', 'Model Predictions', 'Reco X']
print('The first 10 rows and first 10 columns of truevtxx are:', df_sub_true_x_vtx)
print('The first 10 rows and first 10 columns of recovtxx are:', df_sub_reco_x_vtx)
print('The first 10 rows and first 10 columns of concat truevtxx and prediction are:', subset_concat_df)
print('The first 10 rows and first 10 columns of concat truevtxx, prediction and recovtxx are:', subset_concate_df)
np.savetxt("/Users/michaeldolce/Development/files/csv-files/trial_x_modelPred_FHC_abdul.csv", df_true_x_vtx, delimiter=",")

AttributeError: module 'tensorflow.python.distribute.input_lib' has no attribute 'DistributedDatasetInterface'

In [21]:
# Saving the true and reco vtx x values to a csv file

# True info
df_true_x_vtx=pd.DataFrame(true_vtx_x, columns=['True X'])
df_sub_true_x_vtx = df_true_x_vtx.iloc[:10, :10]  # Select the first 10 rows and first 10 columns

# Reco info
df_reco_x_vtx=pd.DataFrame(reco_vtx_x, columns=['Reco X'])
df_sub_reco_x_vtx= df_reco_x_vtx.iloc[:10, :10]

df_E = pd.DataFrame(E, columns=['Energy (GeV)'])
df_pdg = pd.DataFrame(pdg, columns=['PDG code'])
df_CC = pd.DataFrame(test_iscc, columns=['CC'])

df_vars=pd.concat([df_E, df_pdg, df_CC, df_true_x_vtx, df_reco_x_vtx], axis=1)
# df_vars.columns=['True X', 'Reco X'] # in case we want to rename the columns...

# TODO: add the energies and PDGs and CC of the information to the csv file...
# TODO: change isCC to a string instead...
# TODO: and then make plots of them (decide later if they should be a new plotting macro or the existing one from Abdul)

print('head of the pandas df....')
print(df_vars.head())
np.savetxt("/Users/michaeldolce/Development/files/csv-files/x_model_fhc_df_vars.csv", df_vars, delimiter=",")

head of the pandas df....
   Energy (GeV)  PDG code   CC      True X      Reco X
0      2.272811      14.0  0.0   -5.490415   -3.578348
1      1.645375      14.0  1.0   95.721954   94.725662
2      1.756794      14.0  1.0  148.567261  148.680298
3      4.959294      14.0  0.0 -116.638466 -109.047287
4     20.487104     -12.0  1.0  -28.045332  -26.292982
